# 에라스토테네스의 체 효율적으로 구현하기

In [ ]:
import time
import sys
from functools import wraps

# decorator을 사용해서 함수의 실행시간을 측정함
# 어떤 알고리즘이 더 빠른지 벤치마킹 테스트를 할 때 사용함

def multi_elapsed(n):
    def elapsed(f):
        @wraps(f)
        def wrap(*args, **kwargs):
            start_r = time.perf_counter()
            start_p = time.process_time()
            # 함수 실행
            ret = [0]*n
            for i in range(n):
                ret[i] = f(*args, **kwargs)

                if n >= 100:
                    if i % (n//100) == 0:
                        sys.stdout.write(f'\r{f.__name__} : {i // (n//100) + 1}% 진행됨 ')
            end_r = time.perf_counter()
            end_p = time.process_time()
            elapsed_r = end_r - start_r
            elapsed_p = end_p - start_p

            print(f'\n{f.__name__} elapsed: {elapsed_r:.6g}sec (real) / {elapsed_p:.6g}sec (cpu) / {n} repeated')
            return ret
        return wrap
    return elapsed

In [ ]:
# 슬라이싱을 활용해서 여러 값을 동시에 대입하는 이 방식이 많이 효율적임
@multi_elapsed(1000)
def f(m):
    eras = [True]*m
    for i in (2, *range(3, int(m**0.5)+1, 2)): # 2, *(홀수 <= sqrt(m))만 체크
        if eras[i]:
            eras[i*i::2*i] = [False]*( (m - 1 - i*i)//(2*i) + 1 )
    return eras

# 사람들이 보통 구현하는 방식 -> n = 백만(1000000) 기준 새 방식에 비해 5배쯤 느림
@multi_elapsed(1000)
def g(n):
    eras = [True]*n
    for i in (2, *range(3, int(n**0.5)+1, 2)):
        if eras[i]:
            for j in range(i*2, n, i):
                eras[j] = False
    return eras

@multi_elapsed(1000)
def eratosthenes(n):
    MAX = n + 1
    LIM = int(n ** 0.5) + 1
    prime = set(range(1, MAX))
    # TODO: 좀 더 효율적으로 구현하기
    # 5 mod 6 과 1 mod 6을 참으로 설정한다. 이들은 2의배수도 아니고 3의 배수도 아닌 숫자집합이다.
    # 단, 1은 소수가 아니기에 1 mod 6은 7부터 시작한다.
    ret = set(range(5, MAX, 6)) | set(range(7, MAX, 6))
    if n > 2: ret.add(3) # 3 추가
    if n > 1: ret.add(2) # 2 추가
    for i in range(5, LIM, 6):
        # 5 mod 6 부분
        if i in prime:
            prime -= set(range(i * i, MAX, i * 6)) | set(range(i * (i + 2), MAX, i * 6))
        # 1 mod 6 부분
        j = i + 2
        if j in prime:
            prime -= set(range(j * j, MAX, j * 6)) | set(range(j * (j + 4), MAX, j * 6))
    return ret

f(1000000)
g(1000000)
eratosthenes(1000000)